In [1]:
import os

In [2]:
%pwd

'F:\\Question-Answering-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'F:\\Question-Answering-Project'

In [41]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    raw_valid_data_path: Path
    train_data_path: Path
    valid_data_path: Path
    model_path: Path
    #model_config_path: Path
    tokenizer_path: Path
    metric_file_name: Path
    

In [30]:
from Question_Answering.constants import *
from Question_Answering.utils.common import read_yaml, create_directories

In [42]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            raw_valid_data_path = config.raw_valid_data_path,
            train_data_path = config.train_data_path,
            valid_data_path = config.valid_data_path,
            model_path = config.model_path,
            #model_config_path = config.model_config_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name
        )
        return model_evaluation_config

In [32]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
import torch
import pandas as pd
from tqdm.auto import tqdm
import evaluate
from Question_Answering.entity import ModelEvaluationConfig
from collections import defaultdict
import numpy as np
from accelerate import Accelerator
from transformers import default_data_collator
from torch.utils.data import DataLoader

In [43]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def compute_metrics(self, start_logits, end_logits, features, examples):
        example_to_features = defaultdict(list)
        for idx, feature in enumerate(features):
            example_to_features[feature["example_id"]].append(idx)

        n_best = 20
        max_answer_length = 30
        predicted_answers = []
        predicted_answers = []

        for example in tqdm(examples):
            example_id = example["id"]
            context = example["context"]
            answers = []

            # Loop through all features associated with that example
            for feature_index in example_to_features[example_id]:
                start_logit = start_logits[feature_index]
                end_logit = end_logits[feature_index]
                offsets = features[feature_index]["offset_mapping"]

                start_indexes = np.argsort(start_logit)[-1: -n_best - 1: -1].tolist()
                end_indexes = np.argsort(end_logit)[-1: -n_best - 1: -1].tolist()
                for start_index in start_indexes:
                    for end_index in end_indexes:
                        # Skip answers that are not fully in the context
                        if offsets[start_index] is None or offsets[end_index] is None:
                            continue
                        # Skip answers with a length that is either < 0 or > max_answer_length
                        if (
                                end_index < start_index
                                or end_index - start_index + 1 > max_answer_length
                        ):
                            continue

                        answer = {
                            "text": context[offsets[start_index][0]: offsets[end_index][1]],
                            "logit_score": start_logit[start_index] + end_logit[end_index],
                        }
                        answers.append(answer)

            # Select the answer with the best score
            if len(answers) > 0:
                best_answer = max(answers, key=lambda x: x["logit_score"])
                predicted_answers.append(
                    {"id": example_id, "prediction_text": best_answer["text"]}
                )
            else:
                predicted_answers.append({"id": example_id, "prediction_text": ""})

        theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
        return self.metric.compute(predictions=predicted_answers, references=theoretical_answers)

    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = BertTokenizer.from_pretrained(self.config.tokenizer_path)
        #finetuned_config = BertConfig.from_json_file(self.config.model_config_path)
        #model = BertModel.from_pretrained(self.config.model_path, config=finetuned_config).to(device)
        model = AutoModelForQuestionAnswering.from_pretrained(self.config.model_path,trust_remote_code=True).to(device)
        accelerator = Accelerator()

        raw_datasets_validation_split = load_from_disk(self.config.raw_valid_data_path)
        valid_dataset = load_from_disk(self.config.valid_data_path)
        valid_dataset = valid_dataset.remove_columns(["example_id", "offset_mapping"])
        valid_dataset.set_format("torch")

        eval_dataloader = DataLoader(
            valid_dataset,
            shuffle=True,
            collate_fn=default_data_collator,
            batch_size=8
        )


        model.eval()
        start_logits = []
        end_logits = []
        
        for batch in tqdm(eval_dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)
        
            start_logits.append(accelerator.gather(outputs.start_logits).cpu().numpy())
            end_logits.append(accelerator.gather(outputs.end_logits).cpu().numpy())
        start_logits = np.concatenate(start_logits)
        end_logits = np.concatenate(end_logits)
        start_logits = start_logits[: len(valid_dataset)]
        end_logits = end_logits[: len(valid_dataset)]
        
        metric = evaluate.load("squad")
        print(metric.compute())


In [44]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2023-11-04 21:22:43,210: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-04 21:22:43,214: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-04 21:22:43,216: INFO: common: created directory at: artifacts]
[2023-11-04 21:22:43,218: INFO: common: created directory at: artifacts/model_evaluation]


  7%|████████▎                                                                                                              | 95/1353 [08:26<1:51:50,  5.33s/it]


KeyboardInterrupt: 

In [9]:
from safetensors.torch import load_model, save_model

In [15]:
finetuned_model = load_model(model, "artifacts/model_trainer/model.safetensors")

In [14]:
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-cased")

F:\Question-Answering-Project\venv\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dell\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_o

In [27]:
from transformers import BertConfig, BertModel
config = BertConfig.from_json_file("artifacts/model_trainer/config.json")
model = BertModel.from_pretrained("artifacts/model_trainer/model.safetensors", config = config )

Some weights of BertModel were not initialized from the model checkpoint at artifacts/model_trainer/model.safetensors and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:

# Python program to read
# json file
 
import json
 
# Opening JSON file
f = open('artifacts/model_trainer/tokenizer_config.json')
 
# returns JSON object as 
# a dictionary
data = json.load(f)
 
# Iterating through the json
# list
for i in data.keys():
    print(i)
 
# Closing file
f.close()

added_tokens_decoder
clean_up_tokenization_spaces
cls_token
do_lower_case
mask_token
model_max_length
pad_token
sep_token
strip_accents
tokenize_chinese_chars
tokenizer_class
unk_token


In [47]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
import torch
import pandas as pd
from tqdm.auto import tqdm
import evaluate
from Question_Answering.entity import ModelEvaluationConfig
from collections import defaultdict
import numpy as np
from accelerate import Accelerator
from transformers import default_data_collator
from torch.utils.data import DataLoader
from transformers import BertConfig, BertModel
from torch.optim import AdamW

In [70]:
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
finetuned_config = BertConfig.from_json_file("artifacts/model_trainer/config.json")
model = BertModel.from_pretrained("artifacts/model_trainer/model.safetensors", config=finetuned_config).to(device)

raw_datasets_validation_split = load_from_disk("artifacts/data_ingestion/validation")

valid_dataset = load_from_disk("artifacts/data_transformation/validation_dataset")
valid_dataset = valid_dataset.remove_columns(["example_id","offset_mapping"])
valid_dataset.set_format("torch")

eval_dataloader = DataLoader(
    valid_dataset,
    shuffle=True,
    batch_size=8,
)


Some weights of BertModel were not initialized from the model checkpoint at artifacts/model_trainer/model.safetensors and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
eval_dataloader

In [55]:
valid_dataset.column_names

['input_ids',
 'token_type_ids',
 'attention_mask',
 'offset_mapping',
 'example_id']

In [56]:
raw_datasets_validation_split = load_from_disk("artifacts/data_ingestion/validation")

In [57]:
raw_datasets_validation_split.column_names

['id', 'title', 'context', 'question', 'answers']

In [59]:
from tqdm.auto import tqdm
import numpy as np

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [60]:
metric = evaluate.load("squad")

In [ ]:
"""model.eval()
start_logits = []
end_logits = []

for batch in tqdm(eval_dataloader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    """

In [67]:
from accelerate import Accelerator
accelerator = Accelerator()

In [1]:
import evaluate

F:\Question-Answering-Project\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
metric = evaluate.load("squad")


In [3]:
from transformers import BertConfig, BertModel, BertTokenizer

In [5]:
import numpy as np

In [7]:
from transformers import TrainingArguments, Trainer
from transformers import default_data_collator
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from accelerate import Accelerator
from transformers import get_scheduler
from tqdm.auto import tqdm

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
#tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
finetuned_config = BertConfig.from_json_file("/Question-Answering-Project/artifacts/model_trainer/config.json")
model = BertModel.from_pretrained("/Question-Answering-Project/artifacts/model_trainer/model.safetensors").to(device)
tokenizer = BertTokenizer.from_pretrained("/Question-Answering-Project/artifacts/model_trainer/tokenizer_config.json")

raw_datasets_validation_split = load_from_disk("/Question-Answering-Project/artifacts/data_ingestion/validation")
valid_dataset = load_from_disk("/Question-Answering-Project/artifacts/data_transformation/validation_dataset")
valid_dataset = valid_dataset.remove_columns(["example_id", "offset_mapping"])
valid_dataset.set_format("torch")
accelerator = Accelerator()

eval_dataloader = DataLoader(
    valid_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=8
)


model.eval()
start_logits = []
end_logits = []

for batch in tqdm(eval_dataloader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    start_logits.append(accelerator.gather(outputs.start_logits).cpu().numpy())
    end_logits.append(accelerator.gather(outputs.end_logits).cpu().numpy())
start_logits = np.concatenate(start_logits)
end_logits = np.concatenate(end_logits)
start_logits = start_logits[: len(valid_dataset)]
end_logits = end_logits[: len(valid_dataset)]



HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/Question-Answering-Project/artifacts/model_trainer/model.safetensors'. Use `repo_type` argument if needed.

In [46]:
raw_datasets = load_from_disk("artifacts/data_ingestion/validation")
context = raw_datasets[0]["context"]
question = raw_datasets[0]["question"]

In [47]:
context

'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.'

In [48]:
question

'Which NFL team represented the AFC at Super Bowl 50?'